# LLMs e IA Generativa

## Routing con agentes

En este ejercicio se demuestran las capacidades de los agentes mediante el uso de LangChain y LangGraph. A partir del primer ejercicio, se desarrolló una clase denominada RAG, que funciona como una herramienta para los agentes. Se crean dos instancias de RAG, cada una asociada a un índice independiente en Pinecone y vinculada a un CV específico.
Posteriormente, se implementan dos agentes distintos, cada uno con acceso exclusivo a uno de los RAGs previamente definidos. De esta manera, cada agente puede interactuar con su propio conjunto de información y generar respuestas contextualizadas en función del CV persistido en el índice.

El objetivo de este trabajo es simplemente demostrar las capacidades de los agentes, ya que no tiene un fin práctico real.

In [1]:
from RAG import RAG

from typing import TypedDict, Annotated
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, END, START
from pathlib import Path
import os
import operator
from langchain.agents import Tool
from langchain.tools import Tool

from dotenv import load_dotenv

load_dotenv()

True

Se generan dos RAGs, uno para cada CV cargado. En este caso serán dos, uno para el CV de Javier y otro para el CV de Pablo.

In [2]:
# Inicializar la clase RAG
groq_model="llama3-8b-8192"
embedding_model_name="all-MiniLM-L6-v2"
embedding_model_dim=384

# Creo el index correspondiente al resume de Javier
index_name_javier_resume = "javier-resume-embeddings"
rag_javier_resume = RAG(embedding_model_name, embedding_model_dim, index_name_javier_resume, groq_model)
path_javier_resume = Path('javier_resume/')
rag_javier_resume.load_documents(path_javier_resume)

# Creo el index correspondiente al resume de Pablo
index_name_pablo_resume = "pablo-resume-embeddings"
rag_pablo_resume = RAG(embedding_model_name, embedding_model_dim, index_name_pablo_resume, groq_model)
path_pablo_resume = Path('pablo_resume/')
rag_pablo_resume.load_documents(path_pablo_resume)


# Registrar como herramienta al RAG Javier
rag_tool_javier_langchain = Tool(
    name="RAG_Tool_Javier",
    func=rag_javier_resume,
    description="A RAG tool for CV data about Javier Villagra, leveraging Groq hardware and an LLM to retrieve and generate precise answers about individuals' work experience, education, and skills."
)

# Registrar como herramienta al RAG Pablo
rag_tool_pablo_langchain = Tool(
    name="RAG_Tool_Pablo",
    func=rag_pablo_resume,
    description="A RAG tool for CV data about Pablo Segovia, leveraging Groq hardware and an LLM to retrieve and generate precise answers about individuals' work experience, education, and skills."
)


# Inicializar un LLM base
ChatGroq.api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(model_name="llama3-8b-8192")

Se define la clase Agent. Esta misma clase será usada para ambos agentes.

In [3]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_groq)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_groq(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            # Mapero de la respuesta del tool
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

Se generan los dos agentes, uno para cada CV.

In [4]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

javier_bot = Agent(llm, [rag_tool_javier_langchain], system=prompt)

pablo_bot = Agent(llm, [rag_tool_pablo_langchain], system=prompt)

A continuación se genera una clase que actúa como wrapper de los agentes.

In [5]:
class AgentWrapper():

    def __init__(self, javier_bot:Agent, pablo_bot:Agent):
        self.javier_bot = javier_bot
        self.pablo_bot = pablo_bot

    def __get_agent_from_prompt(self, prompt: str) -> Agent:
        prompt_lower = prompt.lower()
        if 'javier' in prompt_lower or 'villagra' in prompt_lower:
            return self.javier_bot
        elif 'pablo' in prompt_lower or 'segovia' in prompt_lower:
            return self.pablo_bot
        else:
            raise KeyError()
        
    def ask(self, message:str) -> str:
        messages = [HumanMessage(content=message)]
        agent = self.__get_agent_from_prompt(message)
        result = agent.graph.invoke({"messages": messages})
        return print(result['messages'][-1].content)



Ahora se realizan consultas sobre ambos CVs.

In [6]:
abot = AgentWrapper(javier_bot, pablo_bot)

abot.ask(message="Does Javier know Java language programming?")

Calling: {'name': 'RAG_Tool_Javier', 'args': {'__arg1': 'Javier Villagra'}, 'id': 'call_ght0', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'RAG_Tool_Javier', 'args': {'__arg1': 'Java'}, 'id': 'call_48yc', 'type': 'tool_call'}
Back to the model!
Calling: {'name': 'RAG_Tool_Javier', 'args': {'__arg1': 'Java'}, 'id': 'call_w9mv', 'type': 'tool_call'}
Back to the model!
Yes, Javier knows Java language programming.


In [7]:
abot.ask(message="Does Pablo know Java language programming?")

Calling: {'name': 'RAG_Tool_Pablo', 'args': {'__arg1': 'Java'}, 'id': 'call_msq2', 'type': 'tool_call'}
Back to the model!
Based on the information provided, it seems that there is no direct connection between Pablo and Java as a programming language. The information available does not mention Java as part of Pablo's work or expertise. Therefore, it can be concluded that Pablo does not know Java language programming.
